In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
from torch.utils.data import random_split
from torchvision.utils import make_grid
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets.utils import download_url
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
from torchvision import models
from torchsummary import summary
from glob import glob
import re
from itertools import compress
import pandas as pd
from torchvision.datasets import STL10
from torchvision.models.feature_extraction import create_feature_extractor
torch.manual_seed(0)
from torchvision.models import resnet18, resnet34

In [ ]:
import logging

import numpy as np
from PIL import Image

import torch
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
from torchvision.models import resnet18, resnet34
from torchvision import transforms
from tqdm import tqdm

In [ ]:
torchvision.__version__

'0.11.1+cu111'

In [ ]:
!git clone https://github.com/samirchar/selfSupervised_fewShot.git
from selfSupervised_fewShot.dataprep import *

Cloning into 'selfSupervised_fewShot'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 28 (delta 0), reused 6 (delta 0), pack-reused 22
Unpacking objects: 100% (28/28), done.


In [ ]:
target_dataset = 'STL10'
source_dataset = 'CIFAR100'

img_size = 32
train_batch_size = 32
batch_size = 512
num_workers = 2
val_size = 5000
full_train_size = 50000 #Could be automatic

source_root = f'simclr_{source_dataset.lower()}'
if not os.path.exists(source_root):
  os.mkdir(source_root)
lincls_path = f'{source_root}/lincls_on_{target_dataset.lower()}'
if not os.path.exists(lincls_path):
  os.mkdir(lincls_path)

In [ ]:
import torch.nn as nn


class SimCLR(nn.Module):
    def __init__(self, base_encoder, projection_dim=128):
        super().__init__()
        self.enc = base_encoder(pretrained=False)  # load model from torchvision.models without pretrained weights.
        self.feature_dim = self.enc.fc.in_features

        # Customize for CIFAR10. Replace conv 7x7 with conv 3x3, and remove first max pooling.
        # See Section B.9 of SimCLR paper.
        self.enc.conv1 = nn.Conv2d(3, 64, 3, 1, 1, bias=False)
        self.enc.maxpool = nn.Identity()
        self.enc.fc = nn.Identity()  # remove final fully connected layer.

        # Add MLP projection.
        self.projection_dim = projection_dim
        self.projector = nn.Sequential(nn.Linear(self.feature_dim, 2048),
                                       nn.ReLU(),
                                       nn.Linear(2048, projection_dim))

    def forward(self, x):
        feature = self.enc(x)
        projection = self.projector(feature)
        return feature, projection

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name):
        self.name = name
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


class CIFAR10Pair(STL10):
    """Generate mini-batche pairs on CIFAR10 training set."""
    def __getitem__(self, idx):
        img, target = self.data[idx], self.targets[idx]
        img = Image.fromarray(img)  # .convert('RGB')
        imgs = [self.transform(img), self.transform(img)]
        return torch.stack(imgs), target  # stack a positive pair


def nt_xent(x, t=0.5):
    x = F.normalize(x, dim=1)
    x_scores =  (x @ x.t()).clamp(min=1e-7)  # normalized cosine similarity scores
    x_scale = x_scores / t   # scale with temperature

    # (2N-1)-way softmax without the score of i-th entry itself.
    # Set the diagonals to be large negative values, which become zeros after softmax.
    x_scale = x_scale - torch.eye(x_scale.size(0)).to(x_scale.device) * 1e5

    # targets 2N elements.
    targets = torch.arange(x.size()[0])
    targets[::2] += 1  # target of 2k element is 2k+1
    targets[1::2] -= 1  # target of 2k+1 element is 2k
    return F.cross_entropy(x_scale, targets.long().to(x_scale.device))


def get_lr(step, total_steps, lr_max, lr_min):
    """Compute learning rate according to cosine annealing schedule."""
    return lr_min + (lr_max - lr_min) * 0.5 * (1 + np.cos(step / total_steps * np.pi))


# color distortion composed by color jittering and color dropping.
# See Section A of SimCLR: https://arxiv.org/abs/2002.05709
def get_color_distortion(s=0.5):  # 0.5 for CIFAR100 by default
    # s is the strength of color distortion
    color_jitter = transforms.ColorJitter(0.8*s, 0.8*s, 0.8*s, 0.2*s)
    rnd_color_jitter = transforms.RandomApply([color_jitter], p=0.8)
    rnd_gray = transforms.RandomGrayscale(p=0.2)
    color_distort = transforms.Compose([rnd_color_jitter, rnd_gray])
    return color_distort


In [ ]:
from torchvision.transforms.transforms import Resize
train_transform = transforms.Compose([transforms.Resize(32),
                                      transforms.RandomResizedCrop(32,scale=(0.2, 1.)),
                                          transforms.RandomHorizontalFlip(),
                                          get_color_distortion(s=0.5),
                                          transforms.ToTensor(),
                                          transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))])
                                        
data_dir =  './data' # get absolute path of data dir

trainset = STL10(root=data_dir,
                            split = "train",
                            transform=train_transform,
                            download=True)

train_loader = DataLoader(trainset,
                              batch_size=512,
                              shuffle=True,
                              num_workers=0,
                              drop_last=True)

  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting ./data/stl10_binary.tar.gz to ./data


In [ ]:
test_transform = transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))])


test_set = STL10(root=data_dir, split="test", transform=test_transform, download=False)

test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [ ]:
class SimCLRLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def nt_xent(self,x,t=0.5):
        x = F.normalize(x, dim=1)
        x_scores =  (x @ x.t()).clamp(min=1e-7)  # normalized cosine similarity scores
        x_scale = x_scores / t   # scale with temperature

        # (2N-1)-way softmax without the score of i-th entry itself.
        # Set the diagonals to be large negative values, which become zeros after softmax.
        x_scale = x_scale - torch.eye(x_scale.size(0)).to(x_scale.device) * 1e5

        # targets 2N elements.
        targets = torch.arange(x.size()[0])
        targets[::2] += 1  # target of 2k element is 2k+1
        targets[1::2] -= 1  # target of 2k+1 element is 2k
        return F.cross_entropy(x_scale, targets.long().to(x_scale.device))

    def forward(self,x,t=0.5):
        return nt_xent(x,t)

In [ ]:
class LinModel(nn.Module):
    """Linear wrapper of encoder."""
    def __init__(self, encoder: nn.Module, feature_dim: int, n_classes: int):
        super().__init__()
        self.enc = encoder
        self.feature_dim = feature_dim
        self.n_classes = n_classes
        self.lin = nn.Linear(self.feature_dim, self.n_classes)

    def forward(self, x):
        return self.lin(self.enc(x))

In [ ]:
backbone = 'resnet18' # or resnet34, resnet50
projection_dim = 128 # "[...] to project the representation to a 128-dimensional latent space"
#Load pretrained model on CIFAR100
device = "cuda" if torch.cuda.is_available() else "cpu"
#resnet18 = resnet18_small(num_classes=100)
base_encoder = eval('resnet18')
model = SimCLR(base_encoder, projection_dim=128).cuda()
model.load_state_dict(torch.load('simclr_best_resnet18.pt'))
criterion = SimCLRLoss()
print(model)
#print(simclr_model)



SimCLR(
  (enc): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): Identity()
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): 

In [ ]:
#  # SimCLR training
#     model.train()
#     optimal_loss = 1e5
#     for epoch in range(1, epochs + 1):
#         loss_meter = AverageMeter("SimCLR_loss")
#         train_bar = tqdm(train_loader)
        
#         for x, y in train_bar:
#             sizes = x.size()
#             x = x.view(sizes[0] * 2, sizes[2], sizes[3], sizes[4]).cuda(non_blocking=True)

#             optimizer.zero_grad()
#             feature, rep = model(x)
#             loss = nt_xent(rep, 0.5)
#             loss.backward()
#             optimizer.step()
#             scheduler.step()

#             loss_meter.update(loss.item(), x.size(0))
#             train_bar.set_description("Train epoch {}, SimCLR loss: {:.4f}".format(epoch, loss_meter.avg))
#         train_loss.append(loss_meter.avg)    
#         if loss_meter.avg < optimal_loss:
#             optimal_loss = loss_meter.avg
#             logger.info("==> New best results")
#             torch.save(model.state_dict(), 'simclr_best_{}.pt'.format(backbone))

In [ ]:
def feature_extractor2(model,layer_name,dataset,device,return_target = True):
  return_nodes = {layer_name:'output'}
  extractor = create_feature_extractor(model,return_nodes)

  extracted_features = []
  targets_list = []
  with torch.no_grad():
    for inputs, targets in dataset:
      inputs, targets = inputs.to(device), targets.to(device)
      sizes = inputs.size()
      inputs = inputs.view(sizes[0] * 2, sizes[2], sizes[3], sizes[4]).cuda(non_blocking=True)
      features = extractor(inputs)
      print("features",features['output'].shape)
      # squeeze 1024 ,512
      extracted_features.append(features['output'].squeeze())
      targets_list.append(targets)

  extracted_features = torch.concat(extracted_features,dim=0)
  targets = torch.concat(targets_list,dim=0)
  
  if return_target:
    return extracted_features.cpu().numpy(),targets.cpu().numpy()
  
  return extracted_features.cpu().numpy()
  #return None, None

In [ ]:
X_train,y_train = feature_extractor(model,'enc.avgpool',train_loader,device)

In [ ]:
X_test,y_test = feature_extractor(model,'enc.avgpool',test_loader,device)

In [ ]:
np.save('X_train_STL.npy',X_train)

np.save('X_test_STL.npy',X_test)

np.save('y_train_STL.npy',y_train)
np.save('y_test_STL.npy',y_test)

In [ ]:
X_train.shape

(4608, 512)

In [ ]:
X_test.shape

(8000, 512)